---

author: Юрій Клебан

---

# Підготовка та очистка текстової інформації

Зчитаємо інформацію про стать з попереднього прикладу:

In [3]:
library(janitor)
data <- read.csv("data/badtitled.csv")
data <- clean_names(data)
data <- as.data.frame(data$person_gender)
colnames(data) <- c("gender")
unlist(data)

gender1  gender2  gender3  gender4  gender5  gender6  gender7  gender8 
  "Male"   "   M" "Female"    "Man" "female"  "F    "  "male."      "m" 
 gender9 gender10 gender11 gender12 gender13 
   "Man" "female"  "F    "  "male."      "m"

Схоже, що ці дані насправді мають всього 2 записи, проте у базу даних їх вносили різні люди або вони були зібрані з різних джерел інформації. Це досить поширена проблема у роботі з даними. Особливо коли відбуваєть заміна людей на рочих місцях або перехід на інше програмне забезпечення.

Якщо це буде розглядатися як факторна змінна без будь-якої попередньої обробки, очевидно, що 8, а не 2 класи будуть збережені. Тому завдання полягає в тому, щоб автоматично розпізнавати наведені вище дані про те, чи відноситься кожен елемент до чоловічої чи жіночої статі. У статистичних контекстах класифікацію таких
"безладні" текстові рядки в ряд фіксованих категорій часто називають кодуванням.

Опишемо два взаємодоповнюючих підходи до кодування рядків: `нормалізація` (`string normalization`) рядків і аналіз схожості тексту (`approximate text matching`).

Розглянемо наступні підходи до очистки текстових даних:

    - Видалення пробілів на початку або в кінці
    - Обрізання/збільшення рядків до певної ширини
    – Перетворення у верхній/нижній регістр.
    – Пошук рядків, що містять прості шаблони (підрядки).
    – Апроксимація рядків на основі "відстаней".

Робота з текстом у `R` здійснюється за допомогою пакету `stringr`.

**Видалення пробілів на початку або в кінці** здійснюється за допомогою функції `str_trim()`.

In [5]:
library(stringr)
str_trim(" ostroh academy  ")
str_trim(" ostroh academy ", side = "left")
str_trim(" ostroh academy ", side = "right")

[1] "ostroh academy"

[1] "ostroh academy "

[1] " ostroh academy"

**Обрізання/збільшення рядків до певної ширини** здійснюється за допомогою функції `str_pad()`.

In [6]:
str_pad(57, width = 6, side = "left", pad = 0)

[1] "000057"

In [7]:
str_pad("ostroh", width = 10, side = "right", pad = "_")

[1] "ostroh____"

**Перетворення у верхній/нижній регістр**

In [8]:
text <- "Ostroh Academy!"
toupper(text)
tolower(text)

[1] "OSTROH ACADEMY!"

[1] "ostroh academy!"

**Пошук рядків, що містять прості шаблони (підрядки)**

Скористаємося функцієя `grep()` та `grepl()` для пошуку підрядків у інформації про стать:

In [10]:
grepl("m", data$gender) # Повертає TRUE/FALSE, якщо знахоить входження рядка
grep("m", data$gender) # Повертає номери рядків, по яких є входження

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

[1] FALSE FALSE  TRUE FALSE  TRUE FALSE  TRUE  TRUE FALSE  TRUE FALSE  TRUE
[13]  TRUE

[1]  3  5  7  8 10 12 13

In [11]:
grepl("m", data$gender, ignore.case = TRUE) # не враховує регістр букв
grepl("m", tolower(data$gender))

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

[1]  TRUE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE
[13]  TRUE

[1]  TRUE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE
[13]  TRUE

In [12]:
data$gender
grepl("^m", data$gender, ignore.case = TRUE) # Показує усі збіги, що починаються з вказаної літери

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

[1] "Male"   "   M"   "Female" "Man"    "female" "F    "  "male."  "m"     
 [9] "Man"    "female" "F    "  "male."  "m"

[1]  TRUE FALSE FALSE  TRUE FALSE FALSE  TRUE  TRUE  TRUE FALSE FALSE  TRUE
[13]  TRUE

**Пошук "відстані" між ряжками** - це аналіз рядків на схожіть з визначенням рівня співпадінь.

In [15]:
adist("ao", "ao")
adist("ao", "oa")
adist("ao", "45fb")

0


2


4


Давайте проаналізуємо інформацію про стать з точки зору схожості текстів:

In [18]:
m <- c("male", "female")
adj_m <- adist(data$gender, m)
#adj_m <- adist(tolower(data$gender), m)
#adj_m <- adist(str_trim(tolower(data$gender), side="both"), m)
colnames(adj_m) <- m 
rownames(adj_m) <- data$gender
adj_m

,male,female
Male,0,2
M,3,5
Female,2,0
Man,2,4
female,2,0
F,4,5
male.,1,3
m,3,5
Man,2,4
female,2,0


In [23]:
# Видалимо повтори
adj_m |> as.data.frame() |> dplyr::distinct()

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

,male,female
,<dbl>,<dbl>
Male,0,2
X...M,3,5
Female,2,0
Man,2,4
F....,4,5
male.,1,3


Примінимо інформацію про відстані до "нечистих" даних про стать:

In [24]:
nums <- apply(adj_m, 1, which.min) # Знайдемо найближчі значення
nums

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

Male      M Female    Man female  F      male.      m    Man female  F     
     1      1      2      1      2      1      1      1      1      2      1 
 male.      m 
     1      1

In [25]:
data.frame(initial = data$gender, coded = m[nums]) # FFFFFFFFFFFFFF - проблема!

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

initial,coded
<chr>,<chr>
Male,male
M,male
Female,female
Man,male
female,female
F,male
male.,male
m,male
Man,male


Як альтернативу для знаходження відстаней між рядками можна використовувати функції з бібліотеки `stringdist`.

In [27]:
#install.packages("stringdist")

In [28]:
library(stringdist)
adist("ao", "oa")
stringdist("oa", "ao") # 1, а було 2

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

2


[1] 1

Спробуємо "очистити" дані, які ми отримали з допомогою функції `amatch()`:

In [30]:
nums <- amatch(data$gender,  c("male", "female"), maxDist = 4) # Знайдемо найближчі значення
nums

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

[1]  1  1  2  1  2 NA  1  1  1  2 NA  1  1

In [31]:
data.frame(initial = data$gender, coded = m[nums]) # FFFFFFFFFFFFFF - проблема!

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

initial,coded
<chr>,<chr>
Male,male
M,male
Female,female
Man,male
female,female
F,NA
male.,male
m,male
Man,male


In [34]:
library(dplyr)
data <- data |> mutate(gender = ifelse(gender == "F", "female", gender)) # ????? # Space
data
data <- data |> mutate(gender = ifelse(str_trim(gender) == "F", "female", gender))
data
nums <- amatch(data$gender,  c("male", "female"), maxDist = 4)
data.frame(initial = data$gender, coded = m[nums]) 

gender
<chr>
Male
M
Female
Man
female
female
male.
m
Man


gender
<chr>
Male
M
Female
Man
female
female
male.
m
Man


initial,coded
<chr>,<chr>
Male,male
M,male
Female,female
Man,male
female,female
female,female
male.,male
m,male
Man,male


Місія виконана! Замінимо та збережемо інформацію у файл для майбутніх експериментів по цій темі:

In [35]:
data <- read.csv("data/badtitled.csv")
data <- clean_names(data)
head(data, 2)

,person_age,person_height,person_weight,person_gender,empty
,<int>,<chr>,<dbl>,<chr>,<lgl>
1,23,185,NA,Male,NA
2,41,175,68.3,M,NA


In [36]:
data <- data |> mutate(person_gender = ifelse(str_trim(person_gender) == "F", "female", person_gender))
m <- c("male", "female")
nums <- amatch(data$person_gender, m, maxDist = 4)
data <- data |> mutate(person_gender = m[nums])
data

person_age,person_height,person_weight,person_gender,empty
<int>,<chr>,<dbl>,<chr>,<lgl>
23,185,NA,male,NA
41,175,68.3,male,NA
11,142*,55.4,female,NA
12,NA,48.2,male,NA
54,191,NA,female,NA
32,168,78.0,female,NA
22,NA,54.0,male,NA
21,165,NA,male,NA
14,NA,90.2,male,NA


Замінимо також висоту на числове значення, а не текст:

In [37]:
data <- data |> 
    mutate(person_height = str_remove(data$person_height, pattern = "[*]"))
data

person_age,person_height,person_weight,person_gender,empty
<int>,<chr>,<dbl>,<chr>,<lgl>
23,185,NA,male,NA
41,175,68.3,male,NA
11,142,55.4,female,NA
12,NA,48.2,male,NA
54,191,NA,female,NA
32,168,78.0,female,NA
22,NA,54.0,male,NA
21,165,NA,male,NA
14,NA,90.2,male,NA


In [38]:
data <- data |> mutate(person_height = as.numeric(person_height))
data

person_age,person_height,person_weight,person_gender,empty
<int>,<dbl>,<dbl>,<chr>,<lgl>
23,185,NA,male,NA
41,175,68.3,male,NA
11,142,55.4,female,NA
12,NA,48.2,male,NA
54,191,NA,female,NA
32,168,78.0,female,NA
22,NA,54.0,male,NA
21,165,NA,male,NA
14,NA,90.2,male,NA


In [39]:
write.csv(data, file = "data/cleaned_titled.csv", row.names = F)

---

## Набори даних

1. https://github.com/kleban/r-book-published/tree/main/datasets/untitled.csv
2. https://github.com/kleban/r-book-published/tree/main/datasets/badtitled.csv
3. https://github.com/kleban/r-book-published/tree/main/datasets/cleaned_titled.csv
4. https://github.com/kleban/r-book-published/tree/main/datasets/cleaned_titled2.csv
5. https://github.com/kleban/r-book-published/tree/main/datasets/river_eco.csv

---

## Використані та додаткові джерела

1. [KPMG Virtual Internship](https://www.insidesherpa.com/virtual-internships/m7W4GMqeT3bh9Nb2c)
2. [An introduction to data cleaning with R / Edwin de Jonge, Mark van der Loo, 2013](https://cran.r-project.org/doc/contrib/de_Jonge+van_der_Loo-Introduction_to_data_cleaning_with_R.pdf)
3. [Anomaly Detection in R](datacamp.com/courses/anomaly-detection-in-r)
4. [K-nearest Neighbor: The maths behind it, how it works and an example](https://medium.com/analytics-vidhya/k-nearest-neighbor-the-maths-behind-it-how-it-works-and-an-example-f1de1208546c)
5. [Quantile. Wikipedia](https://en.wikipedia.org/wiki/Quantile)